In [2]:
import numpy as np
import pandas as pd

import astropy
from astropy.stats import sigma_clip
from astropy.io import fits
from astropy.wcs import WCS

from scipy.stats import sigmaclip

from sklearn.neighbors import KNeighborsClassifier, RadiusNeighborsClassifier

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.gridspec as gridspec
import matplotlib.patches as mpatches
import fitClus2D as fc
from astropy.cosmology import LambdaCDM
%matplotlib inline

from PIL import Image

# These cuts get you a 4x4 arcmin square centered on the image center of the fields
img_cuts = {'a2744':[[519,4520],[412,4413]],
            'm0416':[[367,4368],[380,4381]]}
img_files = {'a2744':'Files/abell2744_RGB.fits',
             'm0416':'Files/macs0416_RGB.fits'}

zclus = {'a2744' : 0.308,
         'm0416' : 0.396}
pgals = {'a2744' : [1.689791e-01, 1.965115e+00, 2.0],
        'm0416' : [3.737113e-01, 1.322081e+00, 2.0]}

In [9]:
cluster = 'a2744'
catalog = "Files/" + cluster + "_master_cleaned.csv"

#Set up clusters, not clusters, and unknowns
dzcut=0.03; sigclip=3; sigcut=3; radialcut=np.inf; maglim=23.5; colorcut=1.0
master = pd.read_csv(catalog)
zdat = master['master_z']
valid = master['master_valid']
memberindx = np.where((zdat>zclus[cluster]-dzcut)&(zdat<zclus[cluster]+dzcut)&(valid==1))[0]
z1 = zdat[memberindx]
z2 = sigma_clip(z1,sigma=sigclip)
print(np.mean(z2),np.std(z2))

dz = sigcut*np.std(z2)
print(dz)
ddat = master['master_d']
zdat = master['master_z']
mag814 = master['master_mag814']
mag606 = master['master_mag606']
valid = master['master_valid']
master_cut = master.iloc[np.where((ddat<radialcut)&(~np.isnan(mag606))&(~np.isnan(mag814))&(mag814<=maglim)&(valid == 1))]

zdat   = master_cut['master_z']
nmemindx = np.where((zdat<zclus[cluster]-dz)|(zdat>zclus[cluster]+dz))[0]
memindx = np.where((zdat>zclus[cluster]-dz)&(zdat<zclus[cluster]+dz))[0]
unknownindx = np.where(np.isnan(zdat))[0]

clusterarr = [master_cut.iloc[i] for i in memindx]
ncluster =[master_cut.iloc[i] for i in nmemindx]
unknown = [master_cut.iloc[i] for i in unknownindx]

XclustCM= [[row['master_mag814'], row['master_mag606'] - row['master_mag814']] for row in clusterarr]
XnclustCM = [[row['master_mag814'], row['master_mag606'] - row['master_mag814']] for row in ncluster]
Xcm = np.array(XclustCM + XnclustCM)

Xclustall= [[row['master_mag814'], row['master_mag606'] - row['master_mag814'], row['master_d']] for row in clusterarr]
Xnclustall = [[row['master_mag814'], row['master_mag606'] - row['master_mag814'], row['master_d']] for row in ncluster]
Xall = np.array(Xclustall + Xnclustall)

unknownXcm = [[row['master_mag814'], row['master_mag606'] - row['master_mag814']] for row in unknown]
unknownXcm = np.array(unknownXcm)

y = np.zeros(len(clusterarr) + len(ncluster))
y[:len(clusterarr)] = 1

clustnum = len(clusterarr)
nclustnum = len(ncluster)
print("number of members: {}".format(len(memindx)))
print("Number of non members: {}".format(len(nmemindx)))
print("Number of unknowns: {}".format(len(unknown)))

print(master.iloc[0])

0.3069860850098326 0.007357462290748165
0.022072386872244495
number of members: 94
Number of non members: 46
Number of unknowns: 257
master_RA            3.930226
master_Dec         -30.340092
master_z                  NaN
owers_RA                  NaN
owers_Dec                 NaN
owers_magr                NaN
owers_z                   NaN
owers_zerr                NaN
owers_quality     -2147483648
glass_lab              b'nan'
glass_RA                  NaN
glass_Dec                 NaN
glass_z                   NaN
adeep_lab              b'nan'
adeep_RA                  NaN
adeep_Dec                 NaN
adeep_quality             NaN
adeep_mag606              NaN
adeep_err606              NaN
adeep_mag814              NaN
adeep_err814              NaN
adeep_mag160              NaN
adeep_err160              NaN
adeep_z                   NaN
adeep_zerr                NaN
adeep_zflag               NaN
coe_lab                b'nan'
coe_RA                    NaN
coe_Dec                   N